In [1]:
import pandas as pd
import numpy as np
import sklearn
import sys
import os
import torch
import tqdm
import warnings
warnings.filterwarnings('ignore')

sys.path.append('../')
sys.path.append('../../')

from sklearn.linear_model import LogisticRegression
from fairws.data_util import load_dataset, load_LIFT_embedding, load_LF
from fairws.metrics import exp_eval
from fairws.sbm import get_baseline_pseudolabel, get_sbm_pseudolabel

2023-08-01 23:40:00.029240: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
repetition = 5

# for dataset_name in  ["bank_marketing", "utkface", "hatexplain", "celeba", "civilcomments"]:
for dataset_name in  ["bank_marketing", "utkface", "hatexplain"]:
    use_LIFT_embedding = False # only for adult, bank_marketing
    sbm_diff_threshold = 0.05
    
    result_collection = pd.DataFrame() # to keep results
    
    for seed in range(repetition):
        np.random.seed(seed)
        x_train, y_train, a_train, x_test, y_test, a_test = load_dataset(dataset_name=dataset_name,
                                                                    data_base_path='../data/')
        for ot_type in ["sinkhorn"]:
            cond = f"sbm({ot_type})"

            L = load_LF(dataset_name, data_base_path='../data')
            if use_LIFT_embedding:
                x_embedding_train, x_embedding_test = load_LIFT_embedding(dataset_name=dataset_name,
                                                                            data_base_path='../data/')
                y_train= get_sbm_pseudolabel(L, x_embedding_train, a_train, dataset_name, seed=seed,
                                             ot_type=ot_type, diff_threshold=sbm_diff_threshold,
                                             use_LIFT_embedding=True)

            else:
                y_train= get_sbm_pseudolabel(L, x_train, a_train, dataset_name, seed=seed,
                                             ot_type=ot_type, diff_threshold=sbm_diff_threshold)

            # downstream task
            model = LogisticRegression(random_state=seed)
            model.fit(x_train, y_train)
            y_pred = model.predict(x_test)
            result = exp_eval(y_test, y_pred, a_test, cond=cond)
            result['seed'] = seed
            print(result)
            result_collection = result_collection.append(result, ignore_index=True)
            
    result_collection.to_csv(os.path.join('../rebuttal_results', f'01_{dataset_name}_sinkhorn.csv'))

100%|██████████| 1000/1000 [00:00<00:00, 1866.13epoch/s]


{'condition': 'sbm(sinkhorn)', 'accuracy': 0.8465647001699441, 'fscore': 0.5150895140664962, 'precision': 0.3992862807295797, 'recall': 0.7255043227665706, 'demographic_parity_gap': 0.12193702161312103, 'equal_opportunity_gap': 0.07966578006744385, 'seed': 0}


computing sbm mapping...: 100%|██████████| 7021/7021 [02:11<00:00, 53.30it/s]


SBM (sinkhorn) saved in ../data/bank_marketing/SBM_mapping/bank_marketing_SBM_mapping_sinkhorn_0->1_1.pt, ../data/bank_marketing/SBM_mapping/bank_marketing_SBM_mapping_sinkhorn_0->1_1.pt!


100%|██████████| 1000/1000 [00:00<00:00, 2931.95epoch/s]


...

100%|██████████| 1000/1000 [00:00<00:00, 2613.44epoch/s]


{'condition': 'sbm(sinkhorn)', 'accuracy': 0.610098906819365, 'fscore': 0.6851618327028164, 'precision': 0.6577885391444713, 'recall': 0.7149122807017544, 'demographic_parity_gap': 0.0821344256401062, 'equal_opportunity_gap': 0.043850839138031006, 'seed': 4}


100%|██████████| 1000/1000 [00:00<00:00, 2888.01epoch/s]


{'condition': 'sbm(sinkhorn)', 'accuracy': 0.8720781340229481, 'fscore': 0.8854618618348582, 'precision': 0.9455994622365199, 'recall': 0.8325160635779506, 'demographic_parity_gap': 0.30642226338386536, 'equal_opportunity_gap': 0.1839233636856079, 'seed': 0}


computing sbm mapping...:  40%|████      | 12748/31698 [5:53:19<9:13:20,  1.75s/it] 

In [ ]:
result_collection

In [3]:
result_collection.groupby(['condition'], as_index=False).agg(
                      {'accuracy':['mean','std'],
                      'fscore':['mean','std'], 
                      'precision': ['mean', 'std'],
                      'recall': ['mean', 'std']})

condition  accuracy        fscore     precision        recall    
                      mean std      mean std      mean std      mean std
0  sbm(sinkhorn)  0.872078 NaN  0.885462 NaN  0.945599 NaN  0.832516 NaN

In [ ]:
repetition = 5

for dataset_name in  ["adult", "bank_marketing"]:
    use_LIFT_embedding = True # only for adult, bank_marketing
    sbm_diff_threshold = 0.05
    
    result_collection = pd.DataFrame() # to keep results
    
    for seed in range(repetition):
        np.random.seed(seed)
        x_train, y_train, a_train, x_test, y_test, a_test = load_dataset(dataset_name=dataset_name,
                                                                    data_base_path='../data/')
        for ot_type in ["sinkhorn"]:
            cond = f"sbm({ot_type})"

            L = load_LF(dataset_name, data_base_path='../data')
            if use_LIFT_embedding:
                x_embedding_train, x_embedding_test = load_LIFT_embedding(dataset_name=dataset_name,
                                                                            data_base_path='../data/')
                y_train= get_sbm_pseudolabel(L, x_embedding_train, a_train, dataset_name, seed=seed,
                                             ot_type=ot_type, diff_threshold=sbm_diff_threshold,
                                             use_LIFT_embedding=True)

            else:
                y_train= get_sbm_pseudolabel(L, x_train, a_train, dataset_name, seed=seed,
                                             ot_type=ot_type, diff_threshold=sbm_diff_threshold)

            # downstream task
            model = LogisticRegression(random_state=seed)
            model.fit(x_train, y_train)
            y_pred = model.predict(x_test)
            result = exp_eval(y_test, y_pred, a_test, cond=cond)
            result['seed'] = seed
            print(result)
            result_collection = result_collection.append(result, ignore_index=True)
            
    result_collection.to_csv(os.path.join('../rebuttal_results', f'01_{dataset_name}_embedding_sinkhorn.csv'))

computing sbm mapping...: 100%|██████████| 5448/5448 [29:45<00:00,  3.05it/s]


SBM (sinkhorn) saved in ../data/adult/SBM_mapping/adult_embedding_SBM_mapping_sinkhorn_0->1_0.pt, ../data/adult/SBM_mapping/adult_embedding_SBM_mapping_sinkhorn_0->1_0.pt!


100%|██████████| 1000/1000 [00:00<00:00, 2727.55epoch/s]


{'condition': 'sbm(sinkhorn)', 'accuracy': 0.7862539156071494, 'fscore': 0.45828144458281445, 'precision': 0.5709852598913887, 'recall': 0.3827353094123765, 'demographic_parity_gap': 0.006466612219810486, 'equal_opportunity_gap': 0.16454002261161804, 'seed': 0}


computing sbm mapping...:  31%|███       | 1663/5448 [09:02<20:15,  3.11it/s]